# Interpolate Effective Area FOXSI2

In [539]:
from scipy import interpolate
import csv
import numpy as np

# EffArea Dir
EADir = '/Users/Kamilobu/Desktop/F2EffAreas/'
EAangles = np.insert(np.arange(-9,9+1,2),5,0)
EAenergies = [4.5,  5.5,  6.5,  7.5,  8.5,  9.5, 11. , 13. , 15. , 17. , 19. , 22.5, 27.5]

def loadEA(det):
    mod = 0 if det == 6 else det
    # Read Eff Area Files
    EApanD  = csv.DictReader(open(EADir+'FOXSI2_Module_X-'+str(mod)+'_EA_pan.txt'),
                           fieldnames=EAangles) # Horizontal axis
    EAtiltD = csv.DictReader(open(EADir+'FOXSI2_Module_X-'+str(mod)+'_EA_tilt.txt'),
                           fieldnames=EAangles) # Vertical axis
    # Skip 4 lines of header
    _ = next(EApanD); _ = next(EApanD); _ = next(EApanD); _ = next(EApanD)
    _ = next(EAtiltD); _ = next(EAtiltD); _ = next(EAtiltD); _ = next(EAtiltD)
    # Save to a list :
    EApan, EAtilt = [], []
    for r in EApanD:
        EApan.append(list(r.values()))
    for r in EAtiltD:
        EAtilt.append(list(r.values()))
    # return as an array of floats
    return np.array(EApan).astype(np.float), np.array(EAtilt).astype(np.float)

def EAenergy(det, e_index, Pxi, Pyi) :
    EA6pan, EA6tilt = loadEA(det)
    EA6raw = np.concatenate((EA6pan[e_index], EA6tilt[e_index]))
    Xea = np.concatenate((EAangles,np.zeros(11)))
    Yea = np.concatenate((np.zeros(11),EAangles))
    EAe = interpolate.interp2d(Xea, Yea, EA6raw, kind='cubic')
    return float(EAe(Pxi,Pyi))

def EAi(det, Ei, Pxi, Pyi):
    '''  function that gives the effective area as function of the
            - detector
            - energy of the event
            - heliographic position (x,y) of the event
    '''
    idown = np.where(np.array(EAenergies) > Ei)[0] - 1
    iup   = np.where(np.array(EAenergies) < Ei)[0] + 1
    indices = []
    for i, j in [(i,j) for i in idown for j in iup] : 
        if i ==j : indices.append(i)
    return np.interp(Ei, (EAenergies[indices[0]], EAenergies[indices[1]]),
                     (EAenergy(det,indices[0],Pxi, Pyi), EAenergy(det,indices[1],Pxi, Pyi)))

In [540]:
# Let's suppose a position and energy for testsing
det, Ei, Pxi, Pyi = 6, 8.4, 5.5, -3.2 # keV, arcmin, arcmin

In [541]:
# det, Ei, Pxi, Pyi
EAi(6, 8.4, 5.5, -3.2)

15.934218099689115

In [544]:
EAi(5, 8.4, 0.0, -12.1)

7.607526983126713